# Import all libraries

In [1]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

from SOM import SOM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

### Generate the dataset (300x1) x 3 Classes

In [2]:
fst_class = 300
snd_class = 300
thd_class = 300

X_1 = np.c_[(np.full(fst_class, 1) + np.random.randn(fst_class),
            np.full(fst_class, 1) + np.random.randn(fst_class))]

X_2 = np.c_[(np.full(snd_class, 1) + np.random.randn(snd_class),
            np.full(snd_class, 16) + np.random.randn(snd_class))]

X_3 = np.c_[(np.full(thd_class, 16) + np.random.randn(thd_class),
            np.full(thd_class, 16) + np.random.randn(thd_class))]

Y_1 = np.full(fst_class, 0)
Y_2 = np.full(snd_class, 1)
Y_3 = np.full(thd_class, 2)

In [3]:
plt.figure(figsize=(10, 7))
plt.scatter(X_1[:, 0], X_1[:, 1], marker='o', c='g')
plt.scatter(X_2[:, 0], X_2[:, 1], marker='o', c='r')
plt.scatter(X_3[:, 0], X_3[:, 1], marker='o', c='b')
plt.style.use('dark_background')
plt.show()

<IPython.core.display.Javascript object>

### Preprocess and shuffle data:

In [4]:
a = np.vstack((X_1, X_2, X_3))
b = np.hstack((Y_1, Y_2, Y_3))
all = np.hstack((a, b.reshape(a.shape[0], 1)))
np.random.shuffle(all)

X = all[:, :2]
Y = all[:, 2]

### Split in train and test:

In [5]:
from sklearn import preprocessing

X_train, X_test, Y_train, Y_test_ = train_test_split(X, Y, train_size=0.7)

X_train = preprocessing.minmax_scale(X_train.reshape(X_train.shape[0], 2), axis=0)
Y_train = preprocessing.minmax_scale(Y_train.reshape(Y_train.shape[0], 1), axis=0)
X_test = preprocessing.minmax_scale(X_test.reshape(X_test.shape[0], 2), axis=0)
Y_test = preprocessing.minmax_scale(Y_test_.reshape(Y_test_.shape[0], 1), axis=0)


# Create, configure and train the model

In [6]:
import time
som = SOM(features_num=2, map_size=(10, 10))

fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()
ax.clear()



for index, individ in enumerate(X_train):
    som.train(individ, lr=0.1, iter=50, radius=2)

    print(f"\rTag: {Y_train[index][0]}", end="")
    ax.imshow(som.win_matrix * -1, cmap='plasma')
    fig.canvas.draw()
    ax.clear()
    if index >= 600:
        time.sleep(1)

<IPython.core.display.Javascript object>

Tag: 1.0

KeyboardInterrupt: 

### Show the maps:

In [ ]:
plt.figure(figsize=(10, 7))

plt.subplot(1, 3, 1, title='Class 1')
plt.imshow(som.predict(np.array([1, 1]) * -1), cmap='plasma')
plt.subplot(1, 3, 2, title='Class 2')
plt.imshow(som.predict(np.array([1, 16]) * -1), cmap='plasma')
plt.subplot(1, 3, 3, title='Class 3')
plt.imshow(som.predict(np.array([16, 16]) * -1), cmap='plasma')
plt.show()

### Check metrics:

In [ ]:
predictions = np.argmax(som.predict(X_test), axis=1)
print(f"MSE: {mse(preprocessing.minmax_scale(predictions), Y_test_)}")

# Visualization

In [ ]:
plt.figure(figsize=(30, 30))


input_plt = plt.subplot2grid((3, 3), (0, 0))
out_plt = plt.subplot2grid((3, 3), (0, 1))

input_plt.scatter(X_1[:, 0], X_1[:, 1], marker='o', c='g')
input_plt.scatter(X_2[:, 0], X_2[:, 1], marker='o', c='r')
input_plt.scatter(X_3[:, 0], X_3[:, 1], marker='o', c='b')

i = 0
for point in X_test:
    if predictions[i] == 0:
        out_plt.scatter(point[0], point[1], marker='x', c='red')
    elif predictions[i] == 1:
        out_plt.scatter(point[0], point[1], marker='x', c='green')
    elif predictions[i] == 2:
        out_plt.scatter(point[0], point[1], marker='x', c='blue')

    i += 1

plt.show()